# ClickHouse

## Import libraries and creating connection

In [2]:
#!pip install clickhouse_driver

In [1]:
from clickhouse_driver import Client


user_name = 'hardda_student_a.nikitin-8'
pwd = '0ddbfab04207297e7de7f96e697814e152de44e69f9fd4d8d5bdbb6b6107518a'

# creating connection ClickHouse
client = Client(host='clickhouse.lab.karpov.courses', port=9000,
                user=user_name, password=pwd, database='hardda')

# checking connection
result = client.execute("SELECT * FROM user_dm_events LIMIT 10")

# showing the result
for row in result[0:1]:
    print(row)

(datetime.date(2022, 2, 1), datetime.date(2022, 1, 31), 'android', 'f7411212fd0e2523e126cbfdd3f226c211212', '4beb10e1-aeeb-4c52-acd2-ce1ddbc1fc24b10e1', 22, 11, 3, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0)


## Tasks

### Task 1. 

What is the size of the table `user_dm_events`?

In [6]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
GROUP BY
  database, table
'''

In [7]:
result = client.execute(query)

In [8]:
for row in result:
    print(row)

('hardda', 'user_dm_events', '4.72 GiB')


### Task 2. 

How many rows does the table have?

In [9]:
query = '''
SELECT
  count(*)
FROM 
  user_dm_events
'''

In [10]:
result = client.execute(query)

In [11]:
print(result)

[(55916675,)]


### Task 3. 

On which set of fields is the table partitioned by?

In [12]:
query = '''
SELECT
  table,
  partition_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [13]:
result = client.execute(query)

In [14]:
print(result)

[('user_dm_events', 'event_date')]


### Task 4. 

What set of fields is the primary key (in ClickHouse terms and definitions)?

In [24]:
query = '''
SELECT
  table,
  primary_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [25]:
result = client.execute(query)

In [26]:
print(result)

[('user_dm_events', 'event_date')]


### Task 5. 

What set of fields is the order by key (in ClickHouse terms and definitions)?

In [31]:
query = '''
SELECT
  table,
  sorting_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [32]:
result = client.execute(query)

In [33]:
print(result)

[('user_dm_events', 'event_date')]


### Task 6.

tbc..